In [8]:
import kaggle as kg
import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils import to_categorical

from keras.layers import Input, Dense, BatchNormalization
from keras.models import Model
from keras.optimizers import SGD


2024-07-22 22:36:55.960896: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-22 22:36:55.986385: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-22 22:36:56.190430: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-22 22:36:56.442049: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-22 22:36:56.621761: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

In [7]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

#Set environment variables for this session
os.environ['KAGGLE_USERNAME'] = 'addhyannigam'
os.environ['KAGGLE_KEY'] = 'c2236445611705b12a39948170285a63'

api = KaggleApi()
api.authenticate()

# Use the API to interact with Kaggle datasets

In [9]:
os.environ["KAGGLE_USERNAME"] = "addhyannigam"
os.environ["KAGGLE_KEY"] = "c2236445611705b12a39948170285a63"


In [10]:
kg.api.authenticate()


In [11]:
kg.api.dataset_download_files(dataset="medahmedkrichen/devanagari-handwritten-character-datase",
                              path="dataset",unzip=True)


Dataset URL: https://www.kaggle.com/datasets/medahmedkrichen/devanagari-handwritten-character-datase


In [12]:
def train_test_df(path):

    img_path = list()
    img_label = list()

    for single_class_dir_path in pathlib.Path(path).glob("*"):

        for single_class_img_path in pathlib.Path(single_class_dir_path).glob("*.png"):

            img_path.append(str(single_class_img_path))
            #print(str(single_class_img_path).split("/")[-2].split("_")[-1])
            img_label.append(str(single_class_img_path).split("/")[-2].split("_")[-1])

    return pd.DataFrame(data={"img_path":img_path,"label":img_label})  

In [13]:
train_path = "dataset/DevanagariHandwrittenCharacterDataset/Train"
test_path = "dataset/DevanagariHandwrittenCharacterDataset/Test"

In [14]:
training_data = train_test_df(train_path)
testing_data = train_test_df(test_path)

In [15]:
training_data

,img_path,label
0,dataset/DevanagariHandwrittenCharacterDataset/...,kha
1,dataset/DevanagariHandwrittenCharacterDataset/...,kha
2,dataset/DevanagariHandwrittenCharacterDataset/...,kha
3,dataset/DevanagariHandwrittenCharacterDataset/...,kha
4,dataset/DevanagariHandwrittenCharacterDataset/...,kha
...,...,...
78195,dataset/DevanagariHandwrittenCharacterDataset/...,petchiryakha
78196,dataset/DevanagariHandwrittenCharacterDataset/...,petchiryakha
78197,dataset/DevanagariHandwrittenCharacterDataset/...,petchiryakha
78198,dataset/DevanagariHandwrittenCharacterDataset/...,petchiryakha


In [16]:
testing_data

,img_path,label
0,dataset/DevanagariHandwrittenCharacterDataset/...,kha
1,dataset/DevanagariHandwrittenCharacterDataset/...,kha
2,dataset/DevanagariHandwrittenCharacterDataset/...,kha
3,dataset/DevanagariHandwrittenCharacterDataset/...,kha
4,dataset/DevanagariHandwrittenCharacterDataset/...,kha
...,...,...
13795,dataset/DevanagariHandwrittenCharacterDataset/...,petchiryakha
13796,dataset/DevanagariHandwrittenCharacterDataset/...,petchiryakha
13797,dataset/DevanagariHandwrittenCharacterDataset/...,petchiryakha
13798,dataset/DevanagariHandwrittenCharacterDataset/...,petchiryakha


In [17]:
character2int = dict(zip(training_data["label"].unique(),range(len(training_data["label"].unique()))))


In [18]:
character2int

{'kha': 0,
 'tra': 1,
 'na': 2,
 'chhya': 3,
 'pha': 4,
 'kna': 5,
 '0': 6,
 '8': 7,
 'gya': 8,
 'dha': 9,
 'yna': 10,
 'dhaa': 11,
 'yaw': 12,
 '2': 13,
 'tha': 14,
 'motosaw': 15,
 'daa': 16,
 'gha': 17,
 'waw': 18,
 'da': 19,
 'cha': 20,
 '4': 21,
 'ga': 22,
 'la': 23,
 '9': 24,
 'pa': 25,
 'chha': 26,
 'thaa': 27,
 'bha': 28,
 'tabala': 29,
 '3': 30,
 'taamatar': 31,
 'ra': 32,
 '1': 33,
 '7': 34,
 'ha': 35,
 'adna': 36,
 'ma': 37,
 'ka': 38,
 'jha': 39,
 'ja': 40,
 '5': 41,
 'ba': 42,
 'patalosaw': 43,
 '6': 44,
 'petchiryakha': 45}

In [19]:
training_data["label"].replace(to_replace=character2int.keys(),value=character2int.values(),
                               inplace=True)

/tmp/ipykernel_1408/2698354796.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  training_data["label"].replace(to_replace=character2int.keys(),value=character2int.values(),
/tmp/ipykernel_1408/2698354796.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  training_data["label"].replace(to_replace

In [20]:
training_data

,img_path,label
0,dataset/DevanagariHandwrittenCharacterDataset/...,0
1,dataset/DevanagariHandwrittenCharacterDataset/...,0
2,dataset/DevanagariHandwrittenCharacterDataset/...,0
3,dataset/DevanagariHandwrittenCharacterDataset/...,0
4,dataset/DevanagariHandwrittenCharacterDataset/...,0
...,...,...
78195,dataset/DevanagariHandwrittenCharacterDataset/...,45
78196,dataset/DevanagariHandwrittenCharacterDataset/...,45
78197,dataset/DevanagariHandwrittenCharacterDataset/...,45
78198,dataset/DevanagariHandwrittenCharacterDataset/...,45


In [21]:
testing_data.replace(to_replace=character2int.keys(),value=character2int.values(),
                     inplace=True)

/tmp/ipykernel_1408/3185811566.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  testing_data.replace(to_replace=character2int.keys(),value=character2int.values(),


In [22]:
testing_data

,img_path,label
0,dataset/DevanagariHandwrittenCharacterDataset/...,0
1,dataset/DevanagariHandwrittenCharacterDataset/...,0
2,dataset/DevanagariHandwrittenCharacterDataset/...,0
3,dataset/DevanagariHandwrittenCharacterDataset/...,0
4,dataset/DevanagariHandwrittenCharacterDataset/...,0
...,...,...
13795,dataset/DevanagariHandwrittenCharacterDataset/...,45
13796,dataset/DevanagariHandwrittenCharacterDataset/...,45
13797,dataset/DevanagariHandwrittenCharacterDataset/...,45
13798,dataset/DevanagariHandwrittenCharacterDataset/...,45


In [23]:
Y_true_train = to_categorical(training_data["label"],num_classes=46)

In [24]:
Y_true_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [25]:
Y_true_test = to_categorical(testing_data["label"],num_classes=46)

In [26]:
Y_true_test

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [27]:
def multiclass_dnn():

    input_to_dnn = Input(shape=(1024,))
    first_dense_out = Dense(units=1024,activation="relu")(input_to_dnn)
    second_dense_out = Dense(units=1024,activation="relu")(first_dense_out)
    second_dense_out = BatchNormalization()(second_dense_out)
    output = Dense(units=46,activation="softmax")(second_dense_out)

    return Model(inputs=[input_to_dnn],outputs=[output])

In [28]:
def custom_data_generator(data_df, Y_true, mb_size):

    for time_step in range(data_df.shape[0]//mb_size):
        X_mb = list()

        for img_path in data_df.iloc[time_step*mb_size:(time_step+1)*mb_size,0]:

            img_np_array = plt.imread(img_path)
            reshaped_np_array = img_np_array.reshape(1024,)
            X_mb.append(reshaped_np_array)

        X_mb = np.array(X_mb)
        Y_true_mb = Y_true[time_step*mb_size:(time_step+1)*mb_size]

        yield X_mb, Y_true_mb

In [29]:
epochs = 50
training_data_mb_size = 782
testing_data_mb_size = 138

In [30]:
model = multiclass_dnn()
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 46)             │        47,150 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,150,446 (8.20 MB)

 Trainable params: 2,148,398 (8.20 MB)

 Non-trainable params: 2,048 (8.00 KB)

In [31]:
def loss_fn(Y_true_mb,Y_pred_mb):

    return tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true=Y_true_mb,
                                                                          y_pred=Y_pred_mb))

optimizer = SGD()

In [32]:
@tf.function
def training_step(X_train_mb,Y_true_train_mb):

    with tf.GradientTape() as tape:
            
        Y_pred_train_mb = model(X_train_mb, training=True)
        training_loss = loss_fn(Y_true_train_mb, Y_pred_train_mb)

    grads = tape.gradient(training_loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(Y_true_train_mb,Y_pred_train_mb)

    return training_loss

In [33]:

@tf.function
def testing_forward_pass(X_test_mb,Y_true_test_mb):

    Y_pred_test_mb = model(X_test_mb,training=False)
    testing_loss = loss_fn(Y_true_test_mb,Y_pred_test_mb)
    test_acc_metric.update_state(Y_true_test_mb,Y_pred_test_mb)

    return testing_loss

In [34]:
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

for epoch in range(epochs):

    training_data_generator = custom_data_generator(training_data,Y_true_train,782)

    for time_step, (X_train_mb, Y_true_train_mb) in enumerate(training_data_generator):
        training_loss = training_step(X_train_mb,Y_true_train_mb)

        if (time_step+1) % 50 == 0:
            print("Epoch %d, Time Step %d, Training loss for one mini batch: %.4f"
            % (epoch+1, time_step+1, float(training_loss)))
            
    training_acc = train_acc_metric.result()    
    print("Epoch %d, Training Accuracy: %.2f" % (epoch+1,float(training_acc)))
    train_acc_metric.reset_states()

    testing_data_generator = custom_data_generator(testing_data,Y_true_test,testing_data_mb_size)

    for X_test_mb, Y_true_test_mb in testing_data_generator:
        testing_loss = testing_forward_pass(X_test_mb,Y_true_test_mb)

    print("\nEpoch %d, Testing Loss for last mini batch: %.4f" % (epoch+1,float(testing_loss)))
    testing_acc = test_acc_metric.result()
    print("Epoch %d, Testing Accuracy: %.2f" % (epoch+1,float(testing_acc)))
    test_acc_metric.reset_states()

    print("\n\n")

Epoch 1, Time Step 50, Training loss for one mini batch: 4.3214
Epoch 1, Time Step 100, Training loss for one mini batch: 4.2499
Epoch 1, Training Accuracy: 0.03


AttributeError: 'CategoricalAccuracy' object has no attribute 'reset_states'